In [3]:
import torch
import torch.nn as nn 
import yaml

In [4]:
# Carico i parametri dal file config.yaml
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

T = config['diffusion']['T']

In [9]:
class TimeEmbedding(nn.Module): # La classe eredita da Module
    def __init__(self, dim):
        """
        Inizializza il time-step embedding.
        dim: dimensione dell'embedding
        """
        super().__init__() # Invoco il costruttore della classe nn.Module
        self.dim = dim
        
        # Modulo MLP per post-elaborazione
        self.embedding = nn.Sequential( # Sequential permette di definire una sequenza di layer
        nn.Linear(dim, dim),  # In input ottengo un embedding sinusoidale di dimensione dim, non occorre modificarne la dimensione
        nn.SiLU(),          # Introduco non linearità, necessario per la backpropagation
        nn.Linear(dim, dim) # Proiezione finale
        )
        
    def get_sinusoidal_embedding(self, t):
        """
        Calcola l'embedding sinusoidale per il timestep t.
        t: tensor di timestep
        Ritorna: tensor di forma [batch_size,dim]
        """
        t = t.float()
        
        # Calcolo le frequenze
        half_dim = self.dim // 2 # L'embedding finale avrà dim elementi, con metà dedicati al seno e metà al coseno
        index_i = torch.arange(half_dim, dtype=torch.float32) # Rappresentà l'indice i nella formua 
        index_i = 10000 ** (-index_i / half_dim) # Inverso della formula (denominatore)
        
        # t * index_i
        angles = t[:, None] * freqs[None, :] # Matrice di angoli contenente t * index_i
        
        # Embedding sinusoidale
        emb = torch.cat([torch.sin(angles), torch.cos(angles)], dim=-1)
        return emb
    
    def forward(self,t):
        """
        Trasforma il timestep t in un embedding elaborato
        t : tensore d timestep, dimensione = a batch_size
        Ritorna: embedding 
        """
        emb = self.get_sinusoidal_embedding(t)
        emb = self.embedding(emb)
        return emb

In [10]:
dim = 128
time_emb = TimeEmbedding(dim)